# MoLFormer for Molecule Embedding

The inference code is at MolFORMER file. `CurrentWork\Structure_Represent_Learning\2023_MoLFormer\notebooks\linear_attention_rotary\attention_analysis_rotary_linear.ipynb`。我使用作者提供的可视化部分进行推理，得到了每个分子的token_embedding，我将结果都存进了pkl文件，方便后续使用。
- 目前的难点可能是怎么进行映射，将`(`这种无用的token去掉，得到原子的对应的初始化表示向量。
- 找找看rdkit或者torchdrug里有没有这种方法

## Smile-Molecule Dataset with MolFormer init node embedding

Get the molecule graph

In [1]:
import pickle
from torchdrug import utils

In [2]:
molecule_pkl = '../../../data/dta-datasets/PDBbind/3d_Molecule.pkl'
with utils.smart_open(molecule_pkl, "rb") as fin:
    molecule_list = pickle.load(fin)

In [3]:
molecule_list

[Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=22, num_bond=46),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=13, num_bond=26),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=17, num_bond=36),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=16, num_bond=32),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=6, num_bond=10),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=31, num_bond=66),
 Molecule(num_atom=21, num_bond=44),
 Molecule(num_atom=15, num_bond=32),
 Molecule(num_atom=8, num_bond=14),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=22, num_bond=50),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=10, num_bond=18),
 Molecule(num_atom=7, num_bond=12),
 Molec

In [4]:
len(molecule_list)

19347

In [5]:
mol = molecule_list[0]
mol

Molecule(num_atom=13, num_bond=24)

In [6]:
mol.mol_feature.shape # FP 1024 for one molecule

torch.Size([1024])

In [7]:
mol.edge_feature

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5424],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5424],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5312],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5312],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4729],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.

In [8]:
radis = mol.edge_feature[:, -1]  # bond length
radis

tensor([1.5424, 1.5424, 1.5312, 1.5312, 1.4729, 1.4729, 1.2549, 1.2549, 1.2563,
        1.2563, 1.5300, 1.5300, 1.5310, 1.5310, 1.2450, 1.2450, 1.2551, 1.2551,
        1.3970, 1.3970, 1.2268, 1.2268, 1.3289, 1.3289])

In [9]:
force = 1 / (radis * radis)
force

tensor([0.4204, 0.4204, 0.4265, 0.4265, 0.4610, 0.4610, 0.6350, 0.6350, 0.6336,
        0.6336, 0.4272, 0.4272, 0.4266, 0.4266, 0.6452, 0.6452, 0.6348, 0.6348,
        0.5124, 0.5124, 0.6644, 0.6644, 0.5662, 0.5662])

In [10]:
mol.edge_weight

<bound method Molecule.edge_weight of Molecule(num_atom=13, num_bond=24)>

In [11]:
mol.edge_weight = force

In [12]:
mol.edge_weight

tensor([0.4204, 0.4204, 0.4265, 0.4265, 0.4610, 0.4610, 0.6350, 0.6350, 0.6336,
        0.6336, 0.4272, 0.4272, 0.4266, 0.4266, 0.6452, 0.6452, 0.6348, 0.6348,
        0.5124, 0.5124, 0.6644, 0.6644, 0.5662, 0.5662])

In [13]:
len(mol.edge_weight)

24

### Loading the molformer_embedding

In [16]:
prefix = '../../../data/dta-datasets/PDBbind/pdbbind_molformer_part'
suffix = '.pkl'
embedding_list = []
token_embedding_list = []
token_list = []
for index in range(1,5):
    molformer_pkl = prefix + str(index) + suffix
    print(molformer_pkl)
    with utils.smart_open(molformer_pkl, "rb") as fin:
        embedding_list.extend(pickle.load(fin))
        token_embedding_list.extend(pickle.load(fin))
        token_list.extend(pickle.load(fin))

../../../data/dta-datasets/PDBbind/pdbbind_molformer_part1.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part2.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part3.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part4.pkl


In [18]:
len(embedding_list)

19347

In [19]:
embedding_list[0].shape  # token_size, embedding_size

torch.Size([768])

### **Patch method**

only mol feature

In [20]:
for index in range(len(molecule_list)):
    embedding = embedding_list[index]
    # token = token_list[index]
    cur_mol = molecule_list[index]
    radis = cur_mol.edge_feature[:, -1] 
    cur_mol.edge_weight = 1 / (radis * radis)
    # cur_mol.edge_weight = 1 / cur_mol.edge_feature[:, -1]  # update the edge weight with distance
    # print(cur_mol)
    # new_embedding, wanted_idx, wanted_tokens = filter_unwanted_tokens(embedding, token, ignore_tokens)
    # if (new_embedding.shape[0] != cur_mol.num_node) :
    #     print(index)
    cur_mol.mol_feature = embedding  # update the node_feature with 768 embedding

In [21]:
molecule_list

[Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=22, num_bond=46),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=13, num_bond=26),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=17, num_bond=36),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=16, num_bond=32),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=6, num_bond=10),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=31, num_bond=66),
 Molecule(num_atom=21, num_bond=44),
 Molecule(num_atom=15, num_bond=32),
 Molecule(num_atom=8, num_bond=14),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=22, num_bond=50),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=10, num_bond=18),
 Molecule(num_atom=7, num_bond=12),
 Molec

In [22]:
len(molecule_list)

19347

In [23]:
new_mol = molecule_list[4]
new_mol

Molecule(num_atom=13, num_bond=26)

In [25]:
new_mol.mol_feature.shape

torch.Size([768])

In [26]:
new_mol.edge_weight

tensor([0.3368, 0.3368, 0.3022, 0.3022, 0.5328, 0.5328, 0.5140, 0.5140, 0.5175,
        0.5175, 0.5188, 0.5188, 0.3516, 0.3516, 0.5597, 0.5597, 0.6463, 0.6463,
        0.4072, 0.4072, 0.4005, 0.4005, 0.4725, 0.4725, 0.4675, 0.4675])

In [27]:
pkl_file = '../../../data/dta-datasets/PDBbind/distanceMol_Molecule.pkl'
with utils.smart_open(pkl_file, "wb") as fout:
    pickle.dump(molecule_list, fout)

Datataset Test for Molecule

In [1]:
from torchdrug import datasets, transforms

In [2]:
protein_view = transforms.ProteinView(view="residue", keys="graph1")
dataset = datasets.PDBBind(path="../../../data/dta-datasets/PDBbind/", protein_method="gearnetesm_pocket", drug_method='distanceMol', transform=protein_view)

==================== Using distanceMol drug method and gearnetesm_pocket protein method! ====================


Constructing Dataset from pkl file: 100%|██████████| 19347/19347 [00:00<00:00, 109432.27it/s]


In [3]:
dataset[0]["graph1"]

PackedProtein(batch_size=1, num_atoms=[49], num_bonds=[834], num_residues=[49])

In [4]:
dataset[0]["graph1"].node_feature.shape

torch.Size([49, 1280])

In [ ]:
dataset[0]["graph1"].num_relation

In [4]:
dataset[0]["graph2"].node_feature.shape

torch.Size([52, 67])

In [5]:
dataset[0]["graph2"].edge_weight

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])

In [6]:
dataset[555]["graph2"].edge_feature

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4868],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4868],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.6200],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3389],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3652],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3652]])

## 3dRandom Dataset with MolFormer init node embedding

Get the molecule graph

In [1]:
import pickle
from torchdrug import utils

In [2]:
molecule_pkl = '../../../data/dta-datasets/PDBbind/3dRandom_Molecule.pkl'
with utils.smart_open(molecule_pkl, "rb") as fin:
    molecule_list = pickle.load(fin)

In [3]:
molecule_list

[Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=22, num_bond=46),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=13, num_bond=26),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=17, num_bond=36),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=16, num_bond=32),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=6, num_bond=10),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=31, num_bond=66),
 Molecule(num_atom=21, num_bond=44),
 Molecule(num_atom=15, num_bond=32),
 Molecule(num_atom=8, num_bond=14),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=22, num_bond=50),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=10, num_bond=18),
 Molecule(num_atom=7, num_bond=12),
 Molec

In [4]:
len(molecule_list)

19347

In [5]:
mol = molecule_list[0]
mol

Molecule(num_atom=13, num_bond=24)

In [6]:
mol.mol_feature.shape # FP 1024 for one molecule

torch.Size([1024])

In [7]:
mol.edge_feature

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4639],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4639],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4601],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4601],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.4489],
        [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.

### Loading the molformer_embedding

In [8]:
prefix = '../../../data/dta-datasets/PDBbind/pdbbind_molformer_part'
suffix = '.pkl'
embedding_list = []
token_embedding_list = []
token_list = []
for index in range(1,5):
    molformer_pkl = prefix + str(index) + suffix
    print(molformer_pkl)
    with utils.smart_open(molformer_pkl, "rb") as fin:
        embedding_list.extend(pickle.load(fin))
        token_embedding_list.extend(pickle.load(fin))
        token_list.extend(pickle.load(fin))

../../../data/dta-datasets/PDBbind/pdbbind_molformer_part1.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part2.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part3.pkl
../../../data/dta-datasets/PDBbind/pdbbind_molformer_part4.pkl


In [9]:
len(embedding_list)

19347

In [10]:
embedding_list[0].shape  # token_size, embedding_size

torch.Size([768])

### **Patch method**

only mol feature

In [11]:
for index in range(len(molecule_list)):
    embedding = embedding_list[index]
    # token = token_list[index]
    cur_mol = molecule_list[index]
    # radis = cur_mol.edge_feature[:, -1] 
    # cur_mol.edge_weight = 1 / (radis * radis)
    # cur_mol.edge_weight = 1 / cur_mol.edge_feature[:, -1]  # update the edge weight with distance
    # print(cur_mol)
    # new_embedding, wanted_idx, wanted_tokens = filter_unwanted_tokens(embedding, token, ignore_tokens)
    # if (new_embedding.shape[0] != cur_mol.num_node) :
    #     print(index)
    cur_mol.mol_feature = embedding  # update the node_feature with 768 embedding

In [12]:
molecule_list

[Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=22, num_bond=46),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=10, num_bond=20),
 Molecule(num_atom=13, num_bond=26),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=17, num_bond=36),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=16, num_bond=32),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=6, num_bond=10),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=9, num_bond=16),
 Molecule(num_atom=31, num_bond=66),
 Molecule(num_atom=21, num_bond=44),
 Molecule(num_atom=15, num_bond=32),
 Molecule(num_atom=8, num_bond=14),
 Molecule(num_atom=13, num_bond=24),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=11, num_bond=20),
 Molecule(num_atom=22, num_bond=50),
 Molecule(num_atom=12, num_bond=24),
 Molecule(num_atom=23, num_bond=48),
 Molecule(num_atom=10, num_bond=18),
 Molecule(num_atom=7, num_bond=12),
 Molec

In [13]:
len(molecule_list)

19347

In [14]:
new_mol = molecule_list[4]
new_mol

Molecule(num_atom=13, num_bond=26)

In [15]:
new_mol.mol_feature.shape

torch.Size([768])

In [16]:
new_mol.edge_weight

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.])

In [17]:
pkl_file = '../../../data/dta-datasets/PDBbind/3dRandomMol_Molecule.pkl'
with utils.smart_open(pkl_file, "wb") as fout:
    pickle.dump(molecule_list, fout)

Datataset Test for Molecule

In [ ]:
from torchdrug import datasets, transforms

In [ ]:
protein_view = transforms.ProteinView(view="residue", keys="graph1")
dataset = datasets.PDBBind(path="../../../data/dta-datasets/PDBbind/", protein_method="gearnetesm_pocket", drug_method='distanceMol', transform=protein_view)

==================== Using distanceMol drug method and gearnetesm_pocket protein method! ====================


Constructing Dataset from pkl file: 100%|██████████| 19347/19347 [00:00<00:00, 109432.27it/s]


In [ ]:
dataset[0]["graph1"]

PackedProtein(batch_size=1, num_atoms=[49], num_bonds=[834], num_residues=[49])

In [ ]:
dataset[0]["graph1"].node_feature.shape

torch.Size([49, 1280])

In [ ]:
dataset[0]["graph1"].num_relation

In [ ]:
dataset[0]["graph2"].node_feature.shape

torch.Size([52, 67])

In [ ]:
dataset[0]["graph2"].edge_weight

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])

In [ ]:
dataset[555]["graph2"].edge_feature

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4868],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4868],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.6200],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3389],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3652],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.3652]])